# Introdution 

The notebook is a behind-the-scenes look at the data wrangling behind the story on *The Urbanizational Evolution of New York City - From Seed to Apple* presented on https://esbenbl.github.io/


The structure of the notebook is:
1. [Motivation](#Motivation)
2. [Basic Statistics](#base_stats)
3. [Data Analysis](#Data_Analysis)
4. [Genre](#Genre)
5. [Visualization](#Visualization)
6. [Discussion](#Discussion)
7. [Contribution](#Controbution)
8. [References](#References)

# Motivation <a id="Motivation"></a>
- What is your dataset?
- Why did you choose this/these particular dataset(s)?
- What was your goal for the end user's experience?


New York is one of the most renowned and global metropolises in the world, some even call it the Capital of the World. In the 19th century due to waves of immigration and industrialization the city's population exploded, also transforming New York. The transformation of New York City’s physical, social and cultural landscape during the 19th time period shaped the cities identity and influenced the development of urban planning, transportation and social policies that continue to shape the city's urban fabric today. Studying the evolution of New York City's urbanization thus provides a rich historical context for understanding contemporary urban challenges and opportunities, making it an interesting and relevant subject for examination. 

We will examine New York's urbanization through data from NYC Department of City Planning, where we will combine two datasets from the department. The first dataset is of the 800.000 buildings in New York and their placement, and the second shows the characteristics of the land use and geographic data which the buildings are built upon. Thus we get a full dataset on both the year of each individual building as well as the type of building, eg. is it commercial or residential. Furthermore we also consider including more data on neighborhood level from New york maybe regarding the overall evolution of other parameters in New York's neighborhoods such as, crime, income, education, or the overall evolution of the metropolis such as subways, schools, hospitals.  etc. 

# Basic Statistics <a id="base_stats"></a>
- Write about your choices in data cleaning and preprocessing
- Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis.

In [ ]:
import geopandas as gpd
import requests 
import matplotlib.pyplot as plt 
plt.rcParams["font.family"] = "Garamond"
import seaborn as sns
import numpy as np
import pandas as pd
import requests
from scipy.interpolate import interp1d

load data

In [ ]:
# Building footprints from https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh
# Documentation https://github.com/CityOfNewYork/nyc-geo-metadata/blob/master/Metadata/Metadata_BuildingFootprints.md
building_footprints = gpd.read_file("Exam_datasets/Building_Footprints.geojson") 

# NYC Borough GeoJson
new_york_boroughs_map = gpd.read_file("https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/new-york-city-boroughs.geojson")

# PLUTO Data from https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page
columns_subset = ["borough","cd", "latitude", "longitude", 'landuse', "assesstot", "numbldgs",
                  "numfloors", "unitstotal", "bldgarea", "comarea", "resarea", "bbl"]
land_use_dataaset = pd.read_csv("Exam_datasets/pluto_22v3_1.csv")[columns_subset]

#### Clean data 

Clean BUILDING FOOTPRINTS

In [ ]:
# remove missing construction_years
building_footprints = building_footprints[-pd.isna(building_footprints.cnstrct_yr)].copy() 

# To int 
building_footprints.cnstrct_yr = building_footprints.cnstrct_yr.astype("int64")

# Construct bins 
ten_year_bins = [0]+[year for year in range(1899,2029,10)]+[2030]
ten_year_labels = ["Before 1900"] + [str(year)+"s" for year in range(1900,2020,10)] + ["2020s"]

# Into Bins
building_footprints["cnstrct_yr_intervals"] = pd.cut(building_footprints.cnstrct_yr, bins = ten_year_bins, labels = ten_year_labels)

# Align BBL with PLUTOs datatypes 
building_footprints = building_footprints.rename(columns = {"mpluto_bbl":"bbl"})
building_footprints.bbl = building_footprints.bbl.astype("int64")

# Distinguish geometry  
building_footprints["building_geometry"] = building_footprints.geometry.copy()

clean PLUTO land use data 

In [ ]:
# drop nan for PLUTO
land_use_dataaset = land_use_dataaset.dropna().copy() # drop na 
land_use_dataaset.bbl = land_use_dataaset.bbl.astype("int64") # to int 

# Dicts to convert values  
landuse_key = {1:"One & Two Family Building",
                2:"Multi-Family Walk-Up Buildings",
                3:"Multi-Family Elevator Buildings",
                4:"Mixed Residential & Commercial Buildings",
                5:"Commerical & Office Buildings",
                6:"Industrial & Manufacturing Buildings",
                7:"Transportation & Utility",
                8:"Public Facilities & Institutions",
                9:"Open Space & Outdoor Recreation",
                10: "Parking Facilities",
                11:"Vacant Land"}

borough_key = {"BK":"Brooklyn",
               "QN":"Queens",
               "MN":"Manhattan",
               "BX":"Bronx",
               "SI":"Staten Island"}

# Map dicts
land_use_dataaset.borough = land_use_dataaset.borough.apply(lambda x: borough_key[x])
land_use_dataaset["landuse_label"] =  land_use_dataaset.landuse.apply(lambda x: landuse_key[x])

### Merge datasets

In [ ]:
buildings_and_landuse = pd.merge(building_footprints, land_use_dataaset, on = "bbl", how = "outer", indicator = True)

### Sanity Check on Merge 

In [ ]:
only_in_building_footprints = buildings_and_landuse.query("_merge == 'left_only'")
only_in_pluto = buildings_and_landuse.query("_merge == 'right_only'")

In [ ]:
print(f"Buildings not found in Building Footprint: {only_in_pluto.shape[0]}")
only_in_pluto.borough.value_counts().plot.bar(title = "Borough of lots not found in Building Footprints", figsize = (8,4));

In [ ]:
print(f"Buildings not found in PLUTO: {only_in_building_footprints.shape[0]}")
only_in_building_footprints.cnstrct_yr_intervals.value_counts().sort_index().plot.bar(title = "Construction decade of buildings not found in PLUTO", figsize = (8,4));

### Keep only BBL matches

In [ ]:
buildings_and_landuse = buildings_and_landuse.query("_merge == 'both'").copy()
buildings_and_landuse.cnstrct_yr = buildings_and_landuse.cnstrct_yr.astype("int64")

# Dimensions of dataset 
buildings_and_landuse.shape

# Data Analysis <a id="Data_Analysis"></a>
- Describe your data analysis and explain what you've learned about the dataset.
- If relevant, talk about your machine-learning.

# Genre <a id="Genre"></a>
- Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?
- Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?

Overall our website and data story structure will be rather linear, structured, with most in common with the magazine visualization genre described by Segel and Heer. But as we also are concerned with keeping the reader active and engaged, we will also incorporate interactive plots in which the reader can explore and engage within the frames of our data story and magazine style, thus if a reader for example have an interest in a specific neighborhood in New York it will be possible to specifically mouse over that and see that neighborhoods data. 

# Visualizations<a id="Visualizations"></a>
- Explain the visualizations you've chosen.
- Why are they right for the story you want to tell?

# Discussion<a id="Discussion"></a>
- What went well?,
- What is still missing? What could be improved?, Why?

# Contributions<a id="Contributions"></a>
- You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That's what you should explain).
- It is not OK simply to write "All group members contributed equally".

# References <a id="References"></a>
- Make sure that you use references when they're needed and follow academic standards.